# Codec Verification & Comparison - jpegexp-rs

## Overview
This notebook provides comprehensive verification and comparison of `jpegexp-rs` codecs against standard reference implementations. It tests encoding correctness, visual quality, and quantitative metrics.

## Codec Status (Updated 2026-01-02)

| Codec | Grayscale | RGB | Quality |
|-------|-----------|-----|---------|
| JPEG 1 | ✅ Working | ✅ Working | Production ready |
| JPEG-LS | ✅ Working | ⚠️ Not supported | Lossless (MAE=0) |
| JPEG 2000 | ⚠️ Stub | ⚠️ Stub | Not functional |

## Purpose
- **Verify encoding correctness**: Ensure jpegexp-rs produces valid, decodable files
- **Compare visual quality**: Side-by-side visual inspection of decoded images
- **Quantitative analysis**: Calculate PSNR (Peak Signal-to-Noise Ratio) metrics
- **CLI integration**: Demonstrate using the jpegexp-rs CLI from Python

## Codecs Tested
1. **JPEG 1** (Baseline JPEG)
   - Reference: Pillow (PIL)
   - Tests: Grayscale encoding/decoding
   
2. **JPEG-LS** (Lossless JPEG-LS) ✅ **Fixed!**
   - Reference: imagecodecs/CharLS
   - Grayscale 8-bit and 16-bit: **Lossless (MAE=0)**
   - RGB: Not yet supported (sample-interleave limitation)
   
3. **JPEG 2000** (JP2/J2K)
   - Reference: imagecodecs/OpenJPEG
   - Status: Stub implementation, not functional

## Methodology
- Generate synthetic test images (gradients, patterns)
- Encode with jpegexp-rs CLI
- Decode with reference libraries
- Compare visually and quantitatively (PSNR)
- Verify format compatibility


In [ ]:
# ============================================================================
# Setup and Configuration
# ============================================================================

%matplotlib inline

# Standard library imports
import os
import subprocess
from io import BytesIO

# Third-party imports
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import imagecodecs  # Provides JPEG-LS (CharLS) and JPEG 2000 (OpenJPEG) decoders

# Create output directory for generated test files
OUTPUT_DIR = os.path.join("..", "tests", "out", "codec_comparison")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Locate jpegexp-rs binary
# Try multiple possible locations (release/debug, relative/absolute paths)
possible_paths = [
    os.path.join("..", "target", "release", "jpegexp.exe"),
    os.path.join("target", "release", "jpegexp.exe"),
    os.path.join("..", "target", "debug", "jpegexp.exe"),
    os.path.join("target", "debug", "jpegexp.exe")
]
BINARY_PATH = None
for p in possible_paths:
    if os.path.exists(p):
        BINARY_PATH = os.path.abspath(p)
        break

if not BINARY_PATH:
    print(f"WARNING: Binary not found in {possible_paths}.")
    print("Please build it first with: cargo build --release")
else:
    print(f"Using binary: {BINARY_PATH}")


In [ ]:
# ============================================================================
# Utility Functions
# ============================================================================

def generate_gradient(width, height):
    """
    Generate a horizontal grayscale gradient test image.

    Args:
        width: Image width in pixels
        height: Image height in pixels

    Returns:
        numpy.ndarray: 2D grayscale image (uint8, 0-255)
    """
    arr = np.linspace(0, 255, width, dtype=np.uint8)
    img = np.tile(arr, (height, 1))
    return img

def generate_rgb_gradient(width, height):
    """
    Generate an RGB gradient test image with varying color channels.

    Args:
        width: Image width in pixels
        height: Image height in pixels

    Returns:
        numpy.ndarray: 3D RGB image (height, width, 3) with uint8 values
        - Red channel: horizontal gradient (0-255)
        - Green channel: vertical gradient (0-255)
        - Blue channel: constant zero
    """
    x = np.linspace(0, 255, width, dtype=np.uint8)
    y = np.linspace(0, 255, height, dtype=np.uint8)
    xv, yv = np.meshgrid(x, y)

    r = xv  # Horizontal gradient
    g = yv  # Vertical gradient
    b = np.zeros_like(xv)  # Constant zero

    return np.stack([r, g, b], axis=-1)

def run_jpegexp(args):
    """
    Execute jpegexp-rs CLI command and handle output.

    Args:
        args: List of command-line arguments (without binary name)

    Returns:
        subprocess.CompletedProcess: Result of the subprocess call
    """
    if not BINARY_PATH:
        raise RuntimeError("Binary path not set. Cannot run jpegexp command.")

    cmd = [BINARY_PATH] + args
    result = subprocess.run(cmd, capture_output=True, text=True)

    if result.returncode != 0:
        print(f"Command Failed: {' '.join(cmd)}")
        print(f"Error: {result.stderr}")
    else:
        print(f"Success: {' '.join(cmd)}")

    return result


In [ ]:
## Test 1: JPEG 1 (Baseline JPEG) - Grayscale

### Purpose
Test JPEG 1 encoding and verify correctness by decoding with Pillow.

### Test Image
- 256x256 grayscale horizontal gradient
- Encoded with jpegexp-rs CLI
- Decoded with Pillow for verification

### Metrics
- PSNR (Peak Signal-to-Noise Ratio) between original and decoded image
- Visual comparison (side-by-side)

# Generate test image and save as raw file
width, height = 256, 256
original_gray = generate_gradient(width, height)
input_raw = os.path.join(OUTPUT_DIR, "test_gray.raw")
original_gray.tofile(input_raw)

# Output path for encoded JPEG
output_jpg = os.path.join(OUTPUT_DIR, "output_gray.jpg")

# Encode using jpegexp-rs CLI
print("Encoding JPEG 1 Grayscale...")
run_jpegexp(["encode", "-i", input_raw, "-o", output_jpg, "-w", str(width), "-H", str(height), "-c", "jpeg"])

# Verify encoding by decoding with Pillow (reference implementation)
if os.path.exists(output_jpg):
    pil_img = Image.open(output_jpg)
    decoded_pil = np.array(pil_img)

    # Calculate PSNR (Peak Signal-to-Noise Ratio)
    mse = np.mean((original_gray - decoded_pil) ** 2)
    psnr = 10 * np.log10(255**2 / mse) if mse > 0 else float('inf')
    print(f"JPEG 1 Grayscale PSNR: {psnr:.2f} dB")
    print(f"  (Higher is better, >30 dB is acceptable, >40 dB is very good)")

    # Visual comparison
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(original_gray, cmap='gray')
    plt.title("Original")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(decoded_pil, cmap='gray')
    plt.title(f"Decoded (JPEG 1)\nPSNR={psnr:.1f}dB")
    plt.axis('off')

    plt.tight_layout()
    plt.show()
else:
    print("ERROR: Output file missing! Encoding may have failed.")


## Test 2: JPEG-LS - Grayscale ✅

### Status: WORKING (Lossless)

JPEG-LS grayscale encoding is now fully functional with **perfect lossless compression** (MAE=0).

### Test Image
- Same 256x256 grayscale gradient as JPEG 1 test
- Encoded with jpegexp-rs CLI using JPEG-LS codec
- Decoded with imagecodecs (CharLS reference implementation)

### Expected Result
- **Lossless encoding**: Pixel-perfect match (MAE = 0)
- All grayscale images (8-bit and 16-bit) should decode identically to the original

In [ ]:

# Output path for encoded JPEG-LS
output_jls = os.path.join(OUTPUT_DIR, "output_gray.jls")

# Encode using jpegexp-rs CLI with JPEG-LS codec
print("Encoding JPEG-LS Grayscale...")
run_jpegexp(["encode", "-i", input_raw, "-o", output_jls, "-w", str(width), "-H", str(height), "-c", "jpegls"])

# Display file information
print("\nFile information:")
run_jpegexp(["info", "-i", output_jls])

# Inspect file header (first 64 bytes) for debugging
if os.path.exists(output_jls):
    with open(output_jls, 'rb') as f:
        header = f.read(64)
    print(f"\nFile header (first 64 bytes, hex): {header.hex()}")

# Verify encoding by decoding with imagecodecs (CharLS reference implementation)
if os.path.exists(output_jls):
    try:
        # Read file and decode
        with open(output_jls, 'rb') as f:
            data = f.read()

        decoded_ls = imagecodecs.jpegls_decode(data)
        print(f"\nDecoded shape: {decoded_ls.shape}, dtype: {decoded_ls.dtype}")

        if decoded_ls.size == 0 or decoded_ls.shape == (0, 0):
            print("ERROR: ImageCodecs returned empty image. Decoding may have failed.")
        else:
            # Calculate pixel-wise difference
            diff = np.abs(original_gray.astype(int) - decoded_ls.astype(int))

            # Check if encoding was lossless (pixel-perfect match)
            is_lossless = np.all(diff == 0)
            print(f"Lossless encoding: {is_lossless}")

            if not is_lossless:
                print(f"Maximum pixel difference: {np.max(diff)}")
                print(f"Mean pixel difference: {np.mean(diff):.2f}")

            # Visual comparison
            plt.figure(figsize=(12, 5))
            plt.subplot(1, 3, 1)
            plt.imshow(decoded_ls, cmap='gray')
            plt.title("Decoded (JPEG-LS)")
            plt.axis('off')

            plt.subplot(1, 3, 2)
            plt.imshow(diff, cmap='hot')
            plt.title(f"Difference Map\n(Max={np.max(diff)})")
            plt.axis('off')
            plt.colorbar()

            plt.subplot(1, 3, 3)
            plt.imshow(original_gray, cmap='gray')
            plt.title("Original (Reference)")
            plt.axis('off')

            plt.tight_layout()
            plt.show()

    except Exception as e:
        print(f"ERROR: ImageCodecs failed to decode: {e}")
        print("This may indicate an encoding issue or format incompatibility.")


In [ ]:
## Test 3: JPEG-LS - RGB ⚠️

### Status: NOT YET SUPPORTED

RGB JPEG-LS encoding is **not yet implemented**. This is a known limitation.

### Technical Reason
RGB images use **sample-interleave mode** (`InterleaveMode::Sample`) which requires 
specialized triplet processing where all three color components of each pixel are 
processed together. This is different from grayscale where each sample is independent.

CharLS implements this as `triplet<sample_type>` - a tuple of (R, G, B) that is 
predicted and encoded as a unit.

See `src/jpegls/mod.rs` for detailed technical documentation on what would be 
needed to add RGB support.

### Test Image (Expected to Fail)
- 256x256 RGB gradient image
- Red: horizontal gradient, Green: vertical gradient, Blue: constant
original_rgb = generate_rgb_gradient(width, height)
input_rgb = os.path.join(OUTPUT_DIR, "test_rgb.raw")
original_rgb.tofile(input_rgb)
output_rgb_jls = os.path.join(OUTPUT_DIR, "output_rgb.jls")

print("Encoding JPEG-LS RGB...")
run_jpegexp(["encode", "-i", input_rgb, "-o", output_rgb_jls, "-w", str(width), "-H", str(height), "-c", "jpegls", "-n", "3"])

if os.path.exists(output_rgb_jls):
    try:
        decoded_rgb = imagecodecs.imread(output_rgb_jls)
        print(f"Decoded shape: {decoded_rgb.shape}")

        if decoded_rgb.size > 0 and decoded_rgb.shape != (0,0):
             plt.figure(figsize=(10, 5))
             plt.subplot(1, 2, 1); plt.imshow(decoded_rgb); plt.title("Decoded RGB (JPEG-LS)")
             plt.title("Expected to fail currently due to known bitstream issue")
             plt.show()
        else:
             print("Detailed failure: ImageCodecs returned empty array.")

    except Exception as e:
        print(f"RGB Decode failed: {e}")


## Test 4: JPEG 2000 - Grayscale

### Purpose
Test JPEG 2000 encoding with quality control and verify with reference decoder.

### Test Image
- 256x256 grayscale gradient
- Encoded with quality parameter (85)
- Decoded with imagecodecs (OpenJPEG reference implementation)

### Metrics
- PSNR between original and decoded image
- Visual comparison


In [ ]:
# --- JPEG-LS RGB Test ---
original_rgb = generate_rgb_gradient(width, height)
input_rgb = os.path.join(OUTPUT_DIR, "test_rgb.raw")
original_rgb.tofile(input_rgb)
output_rgb_jls = os.path.join(OUTPUT_DIR, "output_rgb.jls")

print("Encoding JPEG-LS RGB...")
run_jpegexp(["encode", "-i", input_rgb, "-o", output_rgb_jls, "-w", str(width), "-H", str(height), "-c", "jpegls", "-n", "3"])

if os.path.exists(output_rgb_jls):
    try:
        decoded_rgb = imagecodecs.imread(output_rgb_jls)
        print(f"Decoded shape: {decoded_rgb.shape}")

        if decoded_rgb.size > 0 and decoded_rgb.shape != (0,0):
             plt.figure(figsize=(10, 5))
             plt.subplot(1, 2, 1); plt.imshow(decoded_rgb); plt.title("Decoded RGB (JPEG-LS)")
             plt.title("Expected to fail currently due to known bitstream issue")
             plt.show()
        else:
             print("Detailed failure: ImageCodecs returned empty array.")

    except Exception as e:
        print(f"RGB Decode failed: {e}")


## JPEG 2000 Tests
JPEG 2000 encoding is now implemented! This section tests encoding with quality control and verifies the output with imagecodecs.

In [ ]:
# --- JPEG 2000 Grayscale Test ---
output_j2k = "output_gray.j2k"

print("Encoding JPEG 2000 Grayscale...")
run_jpegexp(["encode", "-i", input_raw, "-o", output_j2k, "-w", str(width), "-H", str(height), "-c", "j2k", "-n", "1", "-q", "85"])

# Check info
run_jpegexp(["info", "-i", output_j2k])

if os.path.exists(output_j2k):
    try:
        # Verify with imagecodecs
        dec_j2k = imagecodecs.imread(output_j2k)
        print(f"Decoded J2K Shape: {dec_j2k.shape}, dtype: {dec_j2k.dtype}")

        # Calculate PSNR
        mse = np.mean((original_gray.astype(float) - dec_j2k.astype(float)) ** 2)
        psnr = 10 * np.log10(255**2 / mse) if mse > 0 else float('inf')
        print(f"JPEG 2000 Grayscale PSNR: {psnr:.2f} dB")

        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1); plt.imshow(original_gray, cmap='gray'); plt.title("Original")
        plt.subplot(1, 2, 2); plt.imshow(dec_j2k, cmap='gray'); plt.title(f"Decoded J2K (PSNR={psnr:.1f}dB)")
        plt.show()
    except Exception as e:
        print(f"ImageCodecs failed to decode J2K: {e}")
else:
    print("J2K Output file not created!")


In [ ]:
# --- JPEG 2000 RGB Test ---
output_j2k_rgb = os.path.join(OUTPUT_DIR, "output_rgb.j2k")

print("Encoding JPEG 2000 RGB...")
run_jpegexp(["encode", "-i", input_rgb, "-o", output_j2k_rgb, "-w", str(width), "-H", str(height), "-c", "j2k", "-n", "3", "-q", "90"])

if os.path.exists(output_j2k_rgb):
    try:
        dec_j2k_rgb = imagecodecs.imread(output_j2k_rgb)
        print(f"Decoded J2K RGB Shape: {dec_j2k_rgb.shape}")

        # Calculate PSNR
        mse = np.mean((original_rgb.astype(float) - dec_j2k_rgb.astype(float)) ** 2)
        psnr = 10 * np.log10(255**2 / mse) if mse > 0 else float('inf')
        print(f"JPEG 2000 RGB PSNR: {psnr:.2f} dB")

        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1); plt.imshow(original_rgb); plt.title("Original RGB")
        plt.subplot(1, 2, 2); plt.imshow(dec_j2k_rgb); plt.title(f"Decoded J2K RGB (PSNR={psnr:.1f}dB)")
        plt.show()
    except Exception as e:
        print(f"ImageCodecs failed to decode J2K RGB: {e}")
else:
    print("J2K RGB Output file not created!")


In [ ]:
# --- JPEG 2000 Transcoding Test ---
# Transcode from JPEG to J2K
if os.path.exists(output_jpg):
    output_jpg_to_j2k = os.path.join(OUTPUT_DIR, "output_jpg_to_j2k.j2k")

    print("Transcoding JPEG to JPEG 2000...")
    run_jpegexp(["transcode", "-i", output_jpg, "-o", output_jpg_to_j2k, "-c", "j2k", "-q", "90"])

    if os.path.exists(output_jpg_to_j2k):
        try:
            dec_j2k_transcoded = imagecodecs.imread(output_jpg_to_j2k)
            print(f"Transcoded J2K Shape: {dec_j2k_transcoded.shape}")

            # Compare with original
            mse = np.mean((original_gray.astype(float) - dec_j2k_transcoded.astype(float)) ** 2)
            psnr = 10 * np.log10(255**2 / mse) if mse > 0 else float('inf')
            print(f"Transcoded J2K PSNR: {psnr:.2f} dB")

            plt.figure(figsize=(10, 5))
            plt.subplot(1, 2, 1); plt.imshow(original_gray, cmap='gray'); plt.title("Original")
            plt.subplot(1, 2, 2); plt.imshow(dec_j2k_transcoded, cmap='gray'); plt.title(f"J2K Transcoded (PSNR={psnr:.1f}dB)")
            plt.show()
        except Exception as e:
            print(f"Failed to decode transcoded J2K: {e}")
    else:
        print("Transcoded J2K file not created!")
else:
    print("Skipping transcoding test (JPEG file not found - run JPEG 1 test first)")
